In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.pipeline import Pipeline
from warnings import filterwarnings
import matplotlib.pyplot as plt

# todo: explain how to pip install DatasetsEvaluator, point to github page

# todo: put back when done -- update datasets evaluator on pypi and retest -- i added the drop=True
#from DatasetsEvaluator import DatasetsEvaluator as de

# Todo: remove once have pip install
import sys  
sys.path.insert(0, 'C:\python_projects\RotationFeatures_project\RotationFeatures')
from RotationFeatures import RotationFeatures # todo: fix once have pip install

sys.path.insert(0, 'C:\python_projects\DatasetsEvaluator_project\DatasetsEvaluator')
import DatasetsEvaluator as de

filterwarnings('ignore')
np.random.seed(0)

In [2]:
cache_folder = "c:\dataset_cache"
partial_result_folder = "c:\\intermediate_results"

## Constants Used to Control the Tests Below

In [3]:
# These specify how many datasets are used in the tests below. Ideally about 50 to 100 datasets would be used,
# but these may be set lower. Set to 0 to skip tests. 

NUM_DATASETS_CLASSIFICATION_DEFAULT = 100
NUM_DATASETS_REGRESSION_DEFAULT = 100
NUM_DATASETS_CLASSIFICATION_GRID_SEARCH = 50
NUM_DATASETS_REGRESSION_GRID_SEARCH = 50

# Classification Tests -- With Default Parameters

## Define the set of datasets on openml we will test against

In [4]:
datasets_tester = de.DatasetsTester()

matching_datasets = datasets_tester.find_datasets( 
    problem_type = "classification",
    min_num_classes = 2,
    max_num_classes = 20,
    min_num_minority_class = 5,
    max_num_minority_class = np.inf,
    min_num_features = 0,
    max_num_features = np.inf,
    min_num_instances = 500,
    max_num_instances = 5_000,
    min_num_numeric_features = 2,
    max_num_numeric_features = 50,
    min_num_categorical_features=0,
    max_num_categorical_features=50)

## Collect the datasets and run the tests

In [5]:
# todo: get this working, then update datasetsEvaluator & this on github & pypi
datasets_tester.collect_data(max_num_datasets_used=NUM_DATASETS_CLASSIFICATION_DEFAULT, 
                             preview_data=False, 
                             save_local_cache=True,
                             check_local_cache=True,
                             path_local_cache=cache_folder)
 

Reading from local cache: 0, id: 15, name: breast-w
Reading from local cache: 1, id: 29, name: credit-approval
Reading from local cache: 2, id: 31, name: credit-g
Reading from local cache: 3, id: 37, name: diabetes
Reading from local cache: 4, id: 38, name: sick
Reading from local cache: 5, id: 181, name: yeast
Reading from local cache: 6, id: 185, name: baseball
Reading from local cache: 7, id: 311, name: oil_spill
Reading from local cache: 8, id: 451, name: irish
Reading from local cache: 9, id: 470, name: profb
Reading from local cache: 10, id: 679, name: rmftsa_sleepdata
Reading from local cache: 11, id: 715, name: fri_c3_1000_25
Reading from local cache: 12, id: 717, name: rmftsa_ladata
Reading from local cache: 13, id: 723, name: fri_c4_1000_25
Reading from local cache: 14, id: 728, name: analcatdata_supreme
Reading from local cache: 15, id: 737, name: space_ga
Reading from local cache: 16, id: 740, name: fri_c3_1000_10
Reading from local cache: 17, id: 743, name: fri_c1_1000_5
R

In [ ]:
pipe1 = Pipeline([('dt', tree.DecisionTreeClassifier(random_state=0))])
pipe2 = Pipeline([('rota', RotationFeatures()), ('dt', tree.DecisionTreeClassifier(random_state=0))])

# This provides an example using some non-default parameters. 
summary_df = datasets_tester.run_tests(estimators_arr = [
                                        ("DT", "Original Features", "Default", pipe1),
                                        ("DT", "Rotation-based Features", "Default", pipe2)],
                                       num_cv_folds=3,
                                       show_warnings=False,
                                       partial_result_folder=partial_result_folder) 

display(summary_df)

## Summarize the results in a table

In [ ]:
def summarize_results(accuracy_metric, show_std_dev=False):
    if (len(summary_df)==0):
        return
    p = pd.DataFrame(summary_df.groupby(['Model', 'Feature Engineering Description'])[accuracy_metric].mean())
    if (show_std_dev):
        p['Avg. Std dev between folds'] = summary_df.groupby(['Model', 'Feature Engineering Description'])['Std dev between folds'].mean()
    p['Avg. Train-Test Gap'] = summary_df.groupby(['Model', 'Feature Engineering Description'])['Train-Test Gap'].mean()
    p['Avg. Fit Time'] = summary_df.groupby(['Model', 'Feature Engineering Description'])['Fit Time'].mean()
    p['Avg. Complexity'] = summary_df.groupby(['Model', 'Feature Engineering Description'])['Model Complexity'].mean()
    display(p)    

In [ ]:
summarize_results('Avg f1_macro')

## Graph the Results

In [ ]:
def plot_results(summary_df, accuracy_metric):
    if (len(summary_df)==0):
        return
    
    # Collect the set of all combinations of model type and feature engineering. In this example, there
    # should just be the two.
    combinations_df = summary_df.groupby(['Model', 'Feature Engineering Description']).size().reset_index()

    summary_df = summary_df.dropna(subset=[accuracy_metric])

    # Draw a single plot, with a line for each feature engineering description. Along the x-axis we have each
    # dataset ordered by lowest to highest score when using the original features. 
    fig_width = min(len(summary_df)/4, 20)
    fig_width = max(fig_width, 5)
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_width,5))
    for row_idx in range(len(combinations_df)):
        m = combinations_df.iloc[row_idx]['Model']
        h = combinations_df.iloc[row_idx]['Feature Engineering Description']

        # Get the subset of summary_df for the current feature engineering method. 
        if (row_idx==0):
            subset_df_1 = summary_df[(summary_df['Model']==m) & (summary_df['Feature Engineering Description']==h)].sort_values(by=accuracy_metric).reset_index()
            x_coords = subset_df_1.index        
            y_coords = subset_df_1[accuracy_metric]
        else:
            subset_df_2 = summary_df[(summary_df['Model']==m) & (summary_df['Feature Engineering Description']==h)]
            y_coords = []
            for i in range(len(subset_df_1)):
                ds = subset_df_1.iloc[i]['Dataset']
                y_coords.append(subset_df_2[subset_df_2['Dataset']==ds][accuracy_metric])
        ax.plot(x_coords, y_coords, label=m + "(" + h + ")")
    t = subset_df_1['Dataset']
    plt.xticks(range(len(t)), t, size='small', rotation=55)            
    plt.legend()
    plt.title(accuracy_metric + " by dataset")
    plt.show()

In [ ]:
plot_results(summary_df, 'Avg f1_macro')

## 2nd Classification Test -- Holding the tree depth to 5

In [ ]:
pipe1 = Pipeline([('dt', tree.DecisionTreeClassifier(max_depth=5, random_state=0))])
pipe2 = Pipeline([('rota', RotationFeatures()), ('dt', tree.DecisionTreeClassifier(max_depth=5, random_state=0))])

# This provides an example using some non-default parameters. 
summary_df = datasets_tester.run_tests(estimators_arr = [
                                        ("DT", "Original Features", "Default", pipe1),
                                        ("DT", "Rotation-based Features", "Default", pipe2)],
                                       num_cv_folds=3,
                                       show_warnings=False,
                                       starting_point=58,
                                       partial_result_folder=partial_result_folder) 

display(summary_df)


Running test on 100 datastets
Running tests on dataset index: 58, dataset: fri_c1_1000_10
	Running tests with model: DT (Original Features), (Default)
	Running tests with model: DT (Rotation-based Features), (Default)
Running tests on dataset index: 59, dataset: fri_c2_1000_5
	Running tests with model: DT (Original Features), (Default)
	Running tests with model: DT (Rotation-based Features), (Default)
Running tests on dataset index: 60, dataset: fri_c2_1000_10
	Running tests with model: DT (Original Features), (Default)
	Running tests with model: DT (Rotation-based Features), (Default)
Running tests on dataset index: 61, dataset: fri_c1_1000_25
	Running tests with model: DT (Original Features), (Default)
	Running tests with model: DT (Rotation-based Features), (Default)


In [ ]:
summarize_results('Avg f1_macro')

In [ ]:
plot_results(summary_df, 'Avg f1_macro')

# Regression Tests - With Default Parameters

In [ ]:
datasets_tester = de.DatasetsTester()

matching_datasets = datasets_tester.find_datasets( 
    problem_type = "regression",
    min_num_features = 0,
    max_num_features = np.inf,
    min_num_instances = 500,
    max_num_instances = 5_000,
    min_num_numeric_features = 2,
    max_num_numeric_features = 50,
    min_num_categorical_features=0,
    max_num_categorical_features=50)

In [ ]:
datasets_tester.collect_data(max_num_datasets_used=NUM_DATASETS_REGRESSION_DEFAULT, 
                             preview_data=False, 
                             save_local_cache=True,
                             check_local_cache=True,
                             path_local_cache=cache_folder)
 

pipe1 = Pipeline([('dt', tree.DecisionTreeRegressor(random_state=0))])
pipe2 = Pipeline([('rota', RotationFeatures()), ('dt', tree.DecisionTreeRegressor(random_state=0))])

summary_df = datasets_tester.run_tests(estimators_arr = [
                                        ("DT", "Original Features", "Default", pipe1),
                                        ("DT", "Rotation-based Features", "Default", pipe2)],
                                       num_cv_folds=3,
                                       show_warnings=True,
                                       partial_result_folder=partial_result_folder) 

display(summary_df)

In [ ]:
summarize_results('Avg NRMSE')

In [ ]:
plot_results(summary_df, 'Avg NRMSE')

## 2nd Regression Test -- Holding the tree depth to 5

In [ ]:
# remove once working & use the same set of 100 files
# datasets_tester.collect_data(max_num_datasets_used=NUM_DATASETS_REGRESSION_DEFAULT, 
#                              preview_data=False, 
#                              save_local_cache=True,
#                              check_local_cache=True,
#                              path_local_cache=cache_folder)
 
pipe1 = Pipeline([('dt', tree.DecisionTreeRegressor(max_depth=5, random_state=0))])
pipe2 = Pipeline([('rota', RotationFeatures()), ('dt', tree.DecisionTreeRegressor(max_depth=5, random_state=0))])

summary_df = datasets_tester.run_tests(estimators_arr = [
                                        ("DT", "Original Features", "Default", pipe1),
                                        ("DT", "Rotation-based Features", "Default", pipe2)],
                                       num_cv_folds=3,
                                       show_warnings=True,
                                       partial_result_folder=partial_result_folder) 

display(summary_df)

In [ ]:
summarize_results('Avg NRMSE')

In [ ]:
plot_results(summary_df, 'Avg NRMSE')

# Classification Tests -- Using CVGridSearch to determine the hyperparameters

In [ ]:
# As this takes much longer than testing with the default parameters, we test with fewer datasets. Note though,
# run_tests_grid_search() uses CV to evaluate the grid search for the best hyperparameters, it does a train-test 
# split on the data for evaluation, so evaluates the predictions quickly, though with more variability than if
# using CV to evaluate as well. 

# todo: display the best hyperparameters too

datasets_tester = de.DatasetsTester()

matching_datasets = datasets_tester.find_datasets(problem_type = "classification")

datasets_tester.collect_data(max_num_datasets_used=NUM_DATASETS_CLASSIFICATION_GRID_SEARCH, 
                             preview_data=False, 
                             save_local_cache=True,
                             check_local_cache=True,
                             path_local_cache=cache_folder)

orig_parameters = {
     'dt__max_depth': (3,4,5,6,100)
}

rota_parameters = {
    'rota__degree_increment': (3,4,10,15,30), 
#     'dt__max_depth': (3,4,5,6)
}

orig_pipe = Pipeline([('dt', tree.DecisionTreeClassifier())])
rota_pipe = Pipeline([('rota', RotationFeatures()), ('dt', tree.DecisionTreeClassifier())])

# This provides an example using some non-default parameters. 
summary_df = datasets_tester.run_tests_grid_search(
        estimators_arr = [
            ("DT", "Original Features", "", orig_pipe),
            ("DT", "Rotation-based Features", "", rota_pipe)],
        parameters_arr=[orig_parameters, rota_parameters],
        num_cv_folds=3,
        show_warnings=False,
        partial_result_folder=partial_result_folder) 

display(summary_df)


In [ ]:
summarize_results('f1_macro', show_std_dev=False)

In [ ]:
plot_results(summary_df, 'f1_macro')

## Regression Tests -- Using CVGridSearch to determine the hyperparameters

In [ ]:
# As this takes much longer than testing with the default parameters, we test with fewer datasets. Note though,
# run_tests_grid_search() uses CV to evaluate the grid search for the best hyperparameters, it does a train-test 
# split on the data for evaluation, so evaluates the predictions quickly, though with more variability than if
# using CV to evaluate as well. 


# todo: display the best hyperparameters too

datasets_tester = de.DatasetsTester()

matching_datasets = datasets_tester.find_datasets(problem_type = "regression")

datasets_tester.collect_data(max_num_datasets_used=NUM_DATASETS_REGRESSION_GRID_SEARCH, 
                             preview_data=False, 
                             save_local_cache=True,
                             check_local_cache=True,
                             path_local_cache=cache_folder)

orig_parameters = {
     'dt__max_depth': (3,4,5,6)
}

rota_parameters = {
     'rota__degree_increment': (3,4,10,15,30),
     'dt__max_depth': (3,4,5,6)
}

orig_pipe = Pipeline([('dt', tree.DecisionTreeClassifier())])
rota_pipe = Pipeline([('rota', RotationFeatures()), ('dt', tree.DecisionTreeClassifier())])

# This provides an example using some non-default parameters. 
summary_df = datasets_tester.run_tests_grid_search(
        estimators_arr = [
            ("DT", "Original Features", "", orig_pipe),
            ("DT", "Rotation-based Features", "", rota_pipe)],
        parameters_arr=[orig_parameters, rota_parameters],
        num_cv_folds=3,
        show_warnings=False,
        partial_result_folder=partial_result_folder) 

display(summary_df)


In [ ]:
summarize_results('Avg NRMSE')

In [ ]:
plot_results(summary_df, 'Avg NRMSE')